In [3]:
import pandas as pd
import pickle

In [4]:
from GetUtteranceData import *

In [ ]:
# Generates the string necessary for joining on
lbl_data = pd.read_csv('OldData/labeled_utterances.csv')
lbl_data['uids'] = lbl_data.uids.apply(eval).apply(list).apply(sorted)
lbl_data['uids_str'] = lbl_data.uids.apply(str)

In [14]:
utr_data = pd.read_csv('refined_utterances.csv')
utr_data['uids'] = utr_data.uids.apply(eval).apply(list).apply(sorted)
utr_data['uids_str'] = utr_data.uids.apply(str)

In [6]:
# Okay first things first, I need utterance times
cnxn = pymysql.connect(**CONN_INFO)

q = 'select uid, time, endTime from currentUtterance'
utr_times = pd.read_sql(q, cnxn)

cnxn.close()

In [16]:
joined = lbl_data.merge(utr_data, on='uids_str', suffixes=['_old', ''])

unjoined = lbl_data[~lbl_data.uids_str.isin(joined.uids_str)]

In [19]:
# I want to form a cartesian product so I can check every utr row for an intersection
unjoined['key'] = 1
utr_data['key'] = 1

unjoined['uids'] = unjoined.uids.apply(set)
utr_data['uids'] = utr_data.uids.apply(set)

cart_prod = unjoined.merge(utr_data, on='key', suffixes=['_old', ''])

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
# Only keep rows with intersections
idx = cart_prod[['uids_old', 'uids']].apply(lambda row: not row['uids_old'].isdisjoint(row['uids']), axis=1)
cart_prod = cart_prod[idx]

In [22]:
# drop redundant columns
cols = [col for col in joined.columns if '_old' not in col]

joined = joined[cols]
cart_prod = cart_prod[cols]

In [23]:
# concatenate data
out_data = pd.concat([joined, cart_prod])

In [27]:
# Write to a new dataframe
out_data.to_csv('merged_labeled_data.csv', index=False)